# Cartpole DQN

Deep Q-Learning Network with Keras and OpenAI Gym, based on [Keon Kim's code](https://github.com/keon/deep-q-learning/blob/master/dqn.py).

#### Import dependencies

In [19]:
import random
import gym
import numpy as np
from collections import deque
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam, SGD
import os # for creating directories

#### Set parameters

In [14]:
env = gym.make('CartPole-v1') # initialise environment

In [3]:
state_size = env.observation_space.shape[0]
state_size

4

In [4]:
action_size = env.action_space.n
action_size

2

In [5]:
batch_size = 32

In [6]:
n_episodes = 1001 # n games we want agent to play (default 1001)

In [7]:
output_dir = 'model_output/cartpole/'

In [8]:
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

#### Define agent

In [27]:
class DQNAgent:
    def __init__(self, state_size, action_size):
        self.state_size = state_size
        self.action_size = action_size
        self.memory = deque(maxlen=2000) # double-ended queue; acts like list, but elements can be added/removed from either end
        self.gamma = 0.95                # decay or discount rate: enables agent to take into account future actions in addition to the immediate ones, but discounted at this rate
        self.epsilon = 1.0               # exploration rate: how much to act randomly; more initially than later due to epsilon decay
        self.epsilon_decay = 0.995       # decrease number of random explorations as the agent's performance (hopefully) improves over time
        self.epsilon_min = 0.01          # minimum amount of random exploration permitted
        self.learning_rate = 0.001       # rate at which NN adjusts models parameters via SGD to reduce cost 
        self.model = self._build_model() # private method 
    
    def _build_model(self):
        # neural net to approximate Q-value function:
        model = Sequential()
        model.add(Dense(24, input_dim=self.state_size, activation='relu')) # 1st hidden layer; states as input
        model.add(Dense(24, activation='relu'))                            # 2nd hidden layer
        model.add(Dense(self.action_size, activation='linear'))            # 2 actions, so 2 output neurons: 0 and 1 (L/R)
        model.compile(loss='mse', optimizer=SGD(learning_rate=self.learning_rate))
        return model
    
    def remember(self, state, action, reward, next_state, done):
        self.memory.append((state, action, reward, next_state, done)) # list of previous experiences, enabling re-training later

    def act(self, state):
        if np.random.rand() <= self.epsilon:   # if acting randomly, take random action
            return random.randrange(self.action_size)
        act_values = self.model.predict(state, verbose=0) # if not acting randomly, predict reward value based on current state
        return np.argmax(act_values[0])        # pick the action that will give the highest reward (i.e., go left or right?)

    def replay(self, batch_size): # method that trains NN with experiences sampled from memory
        minibatch = random.sample(self.memory, batch_size)         # sample a minibatch from memory
        for state, action, reward, next_state, done in minibatch:  # extract data for each minibatch sample
            target = reward  # if done (boolean whether game ended or not, i.e., whether final state or not), then target = reward
            if not done:     # if not done, then predict future discounted reward
                target = (reward + self.gamma *     # (target) = reward + (discount rate gamma) * 
                          np.amax(self.model.predict(next_state, verbose=0)[0])) # (maximum target Q based on future action a')
            target_f = self.model.predict(state, verbose=0)    # approximately map current state to future discounted reward
            target_f[0][action] = target
            print(state, target_f)
            self.model.fit(state, target_f, epochs=1, verbose=0) # single epoch of training with x=state, y=target_f; fit decreases loss btwn target_f and y_hat
        if self.epsilon > self.epsilon_min:
            self.epsilon *= self.epsilon_decay

    def load(self, name):
        self.model.load_weights(name)

    def save(self, name):
        self.model.save_weights(name)

#### Interact with environment

In [28]:
agent = DQNAgent(state_size, action_size) # initialise agent

In [29]:
done = False
for e in range(n_episodes):   # iterate over new episodes of the game
    state = env.reset()       # reset state at start of each new episode of the game
    state = np.reshape(state[0], [1, state_size])
    
    for time in range(5000):  # time represents a frame of the game; goal is to keep pole upright as long as possible up to range, e.g., 500 or 5000 timesteps

        action = agent.act(state)   # action is either 0 or 1 (move cart left or right); decide on one or other here
        next_state, reward, done, _, _ = env.step(action) # agent interacts with env, gets feedback; 4 state data points, e.g., pole angle, cart position        
        
        reward = reward if not done else -10           # reward +1 for each additional frame with pole upright        
        next_state = np.reshape(next_state, [1, state_size])
        agent.remember(state, action, reward, next_state, done) # remember the previous timestep's state, actions, reward, etc.        
        state = next_state     # set "current state" for upcoming iteration to the current next state        
        if done:               # episode ends if agent drops pole or we reach timestep 5000
            print("episode: {}/{}, score: {}, e: {:.2}" # print the episode's score and agent's epsilon
                  .format(e, n_episodes, time, agent.epsilon))
            break # exit loop
            
    if len(agent.memory) > batch_size:
        agent.replay(batch_size) # train the agent by replaying the experiences of the episode
        
    if e % 50 == 0:
        agent.save(output_dir + "weights_" + '{:04d}'.format(e) + ".hdf5")         

episode: 0/1001, score: 25, e: 1.0
episode: 1/1001, score: 10, e: 1.0
[[ 0.0165385  -0.19096363  0.00935776  0.34361774]] [[0.02050916 1.0003213 ]]


2025-05-12 09:40:12.844628: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:435] Loaded runtime CuDNN library: 8.5.0 but source was compiled with: 8.6.0.  CuDNN library needs to have matching major version and equal or higher minor version. If using a binary install, upgrade your CuDNN library.  If building from sources, make sure the library loaded at runtime is compatible with the version specified during compile configuration.
2025-05-12 09:40:12.845181: W tensorflow/core/framework/op_kernel.cc:1839] OP_REQUIRES failed at xla_ops.cc:624 : FAILED_PRECONDITION: DNN library initialization failed. Look at the errors above for more details.
2025-05-12 09:40:12.847731: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:435] Loaded runtime CuDNN library: 8.5.0 but source was compiled with: 8.6.0.  CuDNN library needs to have matching major version and equal or higher minor version. If using a binary install, upgrade your CuDNN library.  If building from sources, make sur

FailedPreconditionError: Graph execution error:

Detected at node SGD/StatefulPartitionedCall_4 defined at (most recent call last):
  File "/usr/lib/python3.10/runpy.py", line 196, in _run_module_as_main

  File "/usr/lib/python3.10/runpy.py", line 86, in _run_code

  File "/home/marcel/.local/lib/python3.10/site-packages/ipykernel_launcher.py", line 17, in <module>

  File "/home/marcel/.local/lib/python3.10/site-packages/traitlets/config/application.py", line 978, in launch_instance

  File "/home/marcel/.local/lib/python3.10/site-packages/ipykernel/kernelapp.py", line 712, in start

  File "/home/marcel/.local/lib/python3.10/site-packages/tornado/platform/asyncio.py", line 215, in start

  File "/usr/lib/python3.10/asyncio/base_events.py", line 603, in run_forever

  File "/usr/lib/python3.10/asyncio/base_events.py", line 1909, in _run_once

  File "/usr/lib/python3.10/asyncio/events.py", line 80, in _run

  File "/home/marcel/.local/lib/python3.10/site-packages/ipykernel/kernelbase